In [ ]:
# Depression and Emotion Analysis from Bengali Dataset ML techniques with optimization

In [ ]:
#!unzip '/content/drive/MyDrive/Thesis/Thesis Code/Classification_Benchmarks_Benglai_NLP-master.zip' -d '/content/drive/MyDrive/Thesis/Thesis Code/Classification_Benchmarks_Benglai_NLP-master'

<a href="https://colab.research.google.com/github/omar-sharif03/NAACL-SRW-2021/blob/main/ML_Algorithms_NAACL_(Emotion_Data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from keras import models
from keras import layers
from tensorflow.keras.layers import LSTM,GRU
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.preprocessing.text import Tokenizer
np.random.seed(42)

CPU times: user 3.41 s, sys: 559 ms, total: 3.97 s
Wall time: 4.68 s


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install BnPreprocessing

## Reading  data

In [ ]:
#read Data
data =pd.read_excel('/content/drive/MyDrive/Thesis/Shared/DFBA Train Data.xlsx')
data_df = data[:][0:6478]
text_data = data_df["text"]
labels = data_df["label2"]
classes = data_df['class2']
classes.value_counts()

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell E4661 is marked as a date but the serial value 6691519.586805556 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


disappointment    1064
sadness           1029
love               910
neutral            895
approval           575
disapproval        377
disgust            325
anger              235
amusement          222
nervousness        164
fear               150
remorse            146
embarrassment      139
joy                116
confusion          110
surprise            21
Name: class2, dtype: int64

#Two label depressed and  nondepressed

In [ ]:
def make_two_class(data_df):
  tag_list = []
  for i in range(len(data_df)):
      if data_df.label2[i] == 'no': 
        tag = "non-depressed"
        tag_list.append(tag)
      else: 
        tag = "depressed"      
        tag_list.append(tag)   
  data_df['label'] = tag_list 
  return data_df
new_df = make_two_class(data_df)

In [ ]:
new_df.tail(5)

##Two label droping

In [ ]:
def make_two_tag_droping(data_df):
  tag_list = []
  text_list = []
  for i in range(len(data_df)):
      if (data_df.class2[i] == 'anger' or data_df.class2[i] ==  'disapproval' or data_df.class2[i] == 'disgust'): 
        tag = "depressive"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif (data_df.class2[i] == 'fear' or data_df.class2[i] ==  'nervousness' or data_df.class2[i] ==  'embarrassment' or data_df.class2[i] ==  'remorse'): 
        tag = "depressive"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif (data_df.class2[i] == 'sadness' or data_df.class2[i] == 'disappointment'): 
        tag = "depressive"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif(data_df.class2[i] == 'joy' or data_df.class2[i] ==  'love' or data_df.class2[i] ==   'approval'or data_df.class2[i] ==  'amusement' ):
        tag = "non-depressive"      
        tag_list.append(tag)
        text_list.append(data_df.text[i])
  
  new_df = pd.DataFrame(list(zip(text_list, tag_list)), columns = ['text', 'label'])   

  

  return new_df
new_df = make_two_tag_droping(data_df)

In [ ]:
new_df.shape

(5452, 2)

#Droping Four class no, mild, moderate, and severe


In [ ]:
def make_four_label_droping(data_df):
  tag_list = []
  text_list = []
  for i in range(len(data_df)):
      if (data_df.class2[i] == 'anger' or data_df.class2[i] ==  'disapproval' or data_df.class2[i] == 'disgust'): 
        tag = "mild"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif (data_df.class2[i] == 'fear' or data_df.class2[i] ==  'nervousness' or data_df.class2[i] ==  'embarrassment' or data_df.class2[i] ==  'remorse'): 
        tag = "moderate"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif (data_df.class2[i] == 'sadness' or data_df.class2[i] == 'disappointment'): 
        tag = "severe"
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      elif(data_df.class2[i] == 'joy' or data_df.class2[i] ==  'love' or data_df.class2[i] ==   'approval'or data_df.class2[i] ==  'amusement' or data_df.class2[i] ==  'neutral'  ):
        tag = "no"      
        tag_list.append(tag)
        text_list.append(data_df.text[i])
      else:
        pass
  
  new_df = pd.DataFrame(list(zip(text_list, tag_list)), columns = ['text', 'label'])   

  return new_df
new_df = make_four_label_droping(data_df)

In [ ]:
new_df.tail(5)

,text,label
6342,"মানুষ মাত্রই পাপী, আর পাপীদের \nমধ্যে তওবাকারী...",no
6343,"কালি ছাড়া যেমন কলম মুল্যহীন,তেমনি নামাজ ছাড়া...",no
6344,যদি রুপ চাও তাহলে আমি শূন্য । 😢🙏\n❌ যদি ভালোবা...,moderate
6345,🌸-আল্লাহর কাছে-!!🥀\n কোন কিছুর অভাব নাই-!!🦋\n...,no
6346,-মসজিদে নামাজ পড়ার লোক নাই. \n- কিন্তু ফেসবুকে...,mild


In [ ]:
new_df.tail(5)

,text,label
5036,নতুন করে ভালোবাসার চাইতে পুরনো ভালোবাসায় মুগ্ধ...,severe
5037,কত চেষ্টা; দূরত্ব না হোক!,severe
5038,ভালোবাসা আর লালসা। \nভালোবাসায় কখনও আগ্রহ হারা...,no
5039,প্রেমের বিচ্ছেদ যখন প্রচণ্ড একজন সৃজনশীল মানুষ...,severe
5040,অনেকদিন একা হাঁটি না। অনেকদিন মানুষ দেখি না। অ...,moderate


#Six Emotion

In [ ]:
def make_6_emotion(data_df):
  tag_list = []
  for i in range(len(data_df)):
      if (data_df.class2[i] == 'anger' or data_df.class2[i] ==  'disapproval' or data_df.class2[i] == 'disgust'): 
        tag = "anger"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'fear' or data_df.class2[i] ==  'nervousness'): 
        tag = "fear"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'sadness' or data_df.class2[i] == 'disappointment'or data_df.class2[i] ==  'embarrassment' or data_df.class2[i] ==  'remorse'): 
        tag = "sadness"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'surprise' or data_df.class2[i] ==   'confusion'): 
        tag = "surprise"
        tag_list.append(tag)
      elif (data_df.class2[i] ==  'neutral'):

        tag = "neutral"
        tag_list.append(tag)
      else:
        tag = "joy"      
        tag_list.append(tag)   
  data_df['label'] = tag_list 
  return data_df

In [ ]:
new_df = make_6_emotion(data_df)

In [ ]:
new_df.head(10)

In [ ]:
def make_5_emotion(data_df):
  tag_list = []
  for i in range(len(data_df)):
      if (data_df.class2[i] == 'anger' or data_df.class2[i] ==  'disapproval' or data_df.class2[i] == 'disgust'): 
        tag = "anger"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'fear' or data_df.class2[i] ==  'nervousness'): 
        tag = "fear"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'sadness' or data_df.class2[i] == 'disappointment'or data_df.class2[i] ==  'embarrassment' or data_df.class2[i] ==  'remorse'): 
        tag = "sadness"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'surprise' or data_df.class2[i] ==   'confusion' or data_df.class2[i] == 'neutral'): 
        tag = "neutral"
        tag_list.append(tag)

      else:
        tag = "joy"      
        tag_list.append(tag)   
  data_df['emotion'] = tag_list 
  return data_df

In [ ]:
new_df = make_5_emotion(data_df)

In [ ]:
def make_7_emotion(data_df):
  tag_list = []
  for i in range(len(data_df)):
      if (data_df.class2[i] == 'anger' or data_df.class2[i] ==  'disapproval'): 
        tag = "anger"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'disgust'):
        tag = "disgust"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'fear' or data_df.class2[i] ==  'nervousness'): 
        tag = "fear"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'sadness' or data_df.class2[i] == 'disappointment'or data_df.class2[i] ==  'embarrassment' or data_df.class2[i] ==  'remorse'): 
        tag = "sadness"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'surprise' or data_df.class2[i] ==   'confusion'): 
        tag = "surprise"
        tag_list.append(tag)
      elif (data_df.class2[i] == 'neutral'): 
        tag = "neutral"
        tag_list.append(tag)
      else:
        tag = "joy"      
        tag_list.append(tag)   
  data_df['label'] = tag_list 
  return data_df

In [ ]:
new_df = make_7_emotion(data_df)

## Drop Amibous post

In [ ]:

def drop_amibigous_df(data_df):
 

  for i in range(len(data_df)):
      if data_df.class2[i] == 'surprise': 
        data_df.drop(i, axis =0, inplace =True)
        
      elif data_df.class2[i] == 'neutral':
        data_df.drop(i, axis =0, inplace =True)
       
      elif data_df.class2[i] == 'confusion':
        data_df.drop(i, axis =0, inplace =True)
        
   
  return data_df

In [ ]:
data_df = drop_amibigous_df(data_df)

In [ ]:
data_df.shape

(5041, 13)

## Get text & labels from modified data frame

In [ ]:
text_data = new_df["text"]
labels = new_df["label"]


In [ ]:
labels.value_counts()

sadness     2378
joy         1823
anger        937
neutral      895
fear         314
surprise     131
Name: label, dtype: int64

In [ ]:
#Preprocessing
import BnPreprocessing as pp
clean_text_data = text_data.apply(pp.remove_noise)

In [ ]:
clean_text_data

In [ ]:
import BnPreprocessing as pp
s =  'জীবনে কখনো I love You শুনি নাই😫🥺🥺'
s = pp.remove_punc(s)
print(s)
s = pp.remove_digits(s)
print(s)
s = pp.remove_nonBangla(s)
print(s)
s = pp.remove_emoticons(s)
print(s)
s = pp.remove_sw(s)
print(s)

জীবনে কখনো I love You শুনি নাই😫🥺🥺
জীবনে কখনো I love You শুনি নাই😫🥺🥺
জীবনে কখনো    শুনি নাই😫🥺🥺
জীবনে কখনো    শুনি নাই🥺🥺
জীবনে কখনো শুনি নাই🥺🥺


In [ ]:

    Original Facebook Status: জীবনে কখনো I love You শুনি নাই😫🥺🥺 
    Translation: (In this life, I did not hear "I love You")
    Preprocessed text: জীবনে কখনো শুনি নাই 


    Original Facebook Status: জীবনে কখনো I love You শুনি নাই😫🥺🥺 
    Translation: (In this life, I did not hear "I love You")
    Preprocessed text: জীবনে কখনো শুনি নাই 

##Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
labels = label.fit_transform(labels)

## train test split scikit-learn

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(text_data, labels, test_size=0.2, shuffle = True, random_state=42)

In [ ]:
train_data_length = len(train_data)
train_data_length

4361

In [ ]:
train_labels.value_counts()

sadness     1903
joy         1449
anger        750
neutral      722
fear         247
surprise     111
Name: label, dtype: int64

In [ ]:
test_labels.value_counts()

sadness     475
joy         374
anger       187
neutral     173
fear         67
surprise     20
Name: label, dtype: int64

In [ ]:
test_data_length = len(test_data)
test_data_length

1636

##Data Visualization

In [ ]:
# plot training vs validation distribution
plt.subplot(1, 2, 1)
plt.hist(classes[0:train_data_length])
plt.title("Train labels")
plt.subplot(1, 2, 2)
plt.hist(classes[train_data_length:])
plt.title("Test labels")
plt.tight_layout()

In [ ]:
# plot training vs validation distribution
plt.figure(figsize= [13.75, 10], dpi = 1000)
plt.hist(classes[train_data_length:])
plt.title("Test labels")
plt.xlabel("Test Levels")
plt.ylabel("Number of Examples")
plt.savefig('/content/drive/MyDrive/Thesis/Two Level Result/Calsses_Test_levels.eps', format='eps')

Logistic Regression

In [ ]:
text_clf = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,2), use_idf = True, tokenizer=lambda x: x.split(), max_features=15000)),  
        ('Transformer', TfidfTransformer()),
       ('clf', LogisticRegressionCV(class_weight='balanced', max_iter=5000)),
])

text_clf.fit(train_data, train_labels)
predicted = text_clf.predict(test_data)
import pandas
report = classification_report(test_labels, predicted, output_dict=True)
df = pandas.DataFrame(report).transpose().round(decimals=2)
df.to_csv('/content/drive/MyDrive/Thesis/Thesis Code/thesis result/report_lr_drop.csv')
report1 = confusion_matrix(test_labels, predicted)
df = pandas.DataFrame(report1)
df.to_csv('/content/drive/MyDrive/Thesis/Thesis Code/thesis result/confusion_lr_drop.csv')


In [ ]:
text_clf = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,2), use_idf = True, tokenizer=lambda x: x.split(), max_features=15000)),  
        ('Transformer', TfidfTransformer()),
       ('clf', SVC(class_weight='balanced')),
])

text_clf.fit(train_data, train_labels)
predicted = text_clf.predict(test_data)
import pandas
report = classification_report(test_labels, predicted, output_dict=True)
df = pandas.DataFrame(report).transpose().round(decimals=2)
df.to_csv('/content/drive/MyDrive/Thesis/Four Level Result/report_svm__drop.csv')
report1 = confusion_matrix(test_labels, predicted)
#df = pandas.DataFrame(report1)
#df.to_csv('/content/drive/MyDrive/Thesis/Four Level Result/confusion_svm__drop.csv')


## ML Models

In [ ]:
%%time
lr_model = LogisticRegressionCV(class_weight='balanced', max_iter=10000)
dt_model = DecisionTreeClassifier(class_weight='balanced')
rf_model = RandomForestClassifier(class_weight='balanced')

mnb_model = MultinomialNB()
svm_model = SVC(class_weight='balanced')
knn_model = KNeighborsClassifier()
ada_model = AdaBoostClassifier()

model_names = ['Logistic Regression','Decision Tree','Random Forest','Naive Bayes', 'SVM', 'KNN', 'AdaBoost']
ml_models = [lr_model,dt_model,rf_model,mnb_model, svm_model, knn_model, ada_model]

CPU times: user 211 µs, sys: 0 ns, total: 211 µs
Wall time: 219 µs


## ML model train

In [ ]:
%time
def model_train(model, train_data, train_labels):
  pipe = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,2), use_idf = True, tokenizer=lambda x: x.split(), max_features=10000)),  
        ('Transformer', TfidfTransformer()),  
        ('M', model)
    ])
  pipe.fit(train_data, train_labels)
  return pipe

trained_ml_models = []
for i in ml_models:
    tt = model_train(i,train_data, train_labels)
    trained_ml_models.append(tt)

### Performance on test set

In [ ]:
%time
def model_train(model, train_data, train_labels):
  pipe = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,2), use_idf = True, tokenizer=lambda x: x.split(), max_features=10000)),  
        ('Transformer', TfidfTransformer()),  
        ('M', model)
    ])
  pipe.fit(train_data, train_labels)
  return pipe

trained_ml_models = []
for i in ml_models:
    tt = model_train(i,train_data, train_labels)
    trained_ml_models.append(tt)
pr_list = []
re_list = []
f1_list = []

def print_results(md, x_test, y_test, name):
    #print("\n\nClassifier: ", name)
    pred_y = md.predict(x_test)
    true, pred = y_test, pred_y
    # print(confusion_matrix(true,pred))
    # print(classification_report(true,pred))
    # print("\nPrecison : ",precision_score(pred,true, average = 'weighted'))
    # print("\nRecall : ",recall_score(pred,true,  average = 'weighted'))
    # print("\nF1 : ",f1_score(pred,true,  average = 'weighted'))

    pr_list.append((round(precision_score(y_test, pred_y, average='weighted'),4)*100)) 
    re_list.append((round(recall_score(y_test, pred_y, average='weighted'),4)*100))
    f1_list.append((round(f1_score(y_test, pred_y, average='weighted'),4)*100))

for i in range(len(trained_ml_models)):
    md = trained_ml_models[i]
    name = model_names[i]
    print_results(md, test_data, test_labels, name)

performance_matrix = pd.DataFrame({'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list},
                                  index =model_names)
performance_matrix.to_csv('/content/drive/MyDrive/Thesis/Thesis Code/thesis result/10k_.2Test_drop.csv')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs


## Ensemble Train, and Test

##Make Whole list

In [ ]:
whole_data = []

def make_whole_list(L):
  for i in L:
    whole_data.append(i)

make_whole_list(train_data)
make_whole_list(test_data)
print(len(whole_data))

2702


In [ ]:
tf = TfidfVectorizer(ngram_range=(1,2), use_idf = True, tokenizer=lambda x: x.split(),  max_features=15000)
tf.fit_transform(whole_data) ### Bit confusion arises here need to resolve
trained_texts = tf.transform(train_data)
test_texts = tf.transform(test_data)

In [ ]:
%%time
from sklearn.ensemble import VotingClassifier
LogReg_clf = LogisticRegressionCV(class_weight='balanced', max_iter=400)
#rt_clf =   RandomForestClassifier(class_weight='balanced')
SVC_clf = SVC(class_weight='balanced')
#ada_model = AdaBoostClassifier()
voting_clf = VotingClassifier(estimators=[ ('LogReg', LogReg_clf),('SVC', SVC_clf)], voting='hard')
voting_clf.fit(trained_texts, train_labels)


CPU times: user 13.7 s, sys: 14.8 s, total: 28.5 s
Wall time: 15.2 s


## Test 

In [ ]:
preds = voting_clf.predict(test_texts)
true, pred = test_labels, preds
print(confusion_matrix(true,pred))
print(classification_report(true,pred))
print("\nPrecison : ",precision_score(true, pred, average = 'weighted'))
print("\nRecall : ",recall_score(true, pred,  average = 'weighted'))
print("\nF1 : ",f1_score(true, pred,  average = 'weighted'))

[[422   2]
 [115   2]]
                precision    recall  f1-score   support

    depressive       0.79      1.00      0.88       424
non-depressive       0.50      0.02      0.03       117

      accuracy                           0.78       541
     macro avg       0.64      0.51      0.46       541
  weighted avg       0.72      0.78      0.70       541


Precison :  0.7240281980056245

Recall :  0.7837338262476895

F1 :  0.6954649550969024


## Cross-validation ML Model

In [ ]:
# Random Forrest
rf_params = {'criterion': 'entropy', 
             'max_depth': 15, 
             'n_estimators': 22984, 
             'min_samples_leaf': 3, 
             'min_samples_split': 9, 
             'max_leaf_nodes': 239, 
             'random_state': 22}
             {'criterion': 'gini', 'max_depth': 12, 'n_estimators': 19253, 'min_samples_leaf': 2, 'min_samples_split': 13, 'max_leaf_nodes': 112}

In [ ]:
{'criterion': 'gini', 'max_depth': 12, 'n_estimators': 19253, 'min_samples_leaf': 2, 'min_samples_split': 13, 'max_leaf_nodes': 112}

In [ ]:
mlp_params = {'activation': 'relu', 
              'solver': 'lbfgs', 
              'hidden_layer_sizes': 1283, 
              'alpha': 0.3849485717707319, 
              'batch_size': 163, 
              'learning_rate': 'constant',
              'max_iter':1000}

In [ ]:
knn_params = {'weights': 'distance', 
              'n_neighbors': 32}

In [ ]:
lr_params = {'multi_class':'multinomial',
             'class_weight': None, 
             'solver': 'saga', 
             'max_iter':10000}

In [ ]:
models = {
          'rf':RandomForestClassifier(**rf_params), 
          'mlp':MLPClassifier(**mlp_params), 
          'kn':KNeighborsClassifier(**knn_params),  
          'lr':LogisticRegression(**lr_params)}

model_abrv = { 
              'rf':'Random Forest Classifier', 
              'mlp':'MLP Classifier',
              'kn':'K-Nearest Neighbors', 
              'lr':'Logistic Regression', 
              }

In [ ]:
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14, model='clf', save=True):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a seaborn heatmap. 
    Saves confusion matrix file to jpg file."""
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, ax=ax, fmt="d", cmap=plt.cm.Oranges)
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
        
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    # fix for mpl bug that cuts off top/bottom of seaborn viz
    b, t = plt.ylim() 
    b += 0.5 
    t -= 0.5 
    plt.ylim(b, t) 
    if save == True:
        plt.savefig('tuned_' + model_abrv[model] + '_confusion_matrix.jpg')
    plt.show()

In [ ]:
for key in models.keys():
    model(key, cv= True, print_stat = True)  #set cv = True for cross-validation .Required much time

In [ ]:
def model(clf, X_train= train_data, X_test=test_data, y_train=train_labels, y_test= test_labels, models=models, save=False, print_stat=True, inc_train=False, cv=False):
    """Trains models and outputs score metrics. Takes an identifier, list of models, and split dataset as inputs and has options for saving model, 
    printing confusion matrix and classification report and getting cross-validated 5 fold accuracy."""
    
    clf_model = models[clf]
    #clf_model.fit(X_train, y_train)
    y_pred = clf_model.predict(X_test)
    if print_stat == True:
        clf_report = pd.DataFrame(classification_report(y_test,y_pred, output_dict=True)).T
        clf_report.to_csv('tuned_' + model_abrv[clf] + '_classification_report.csv')
        print(model_abrv[clf])
        print('\nTest Stats\n', classification_report(y_test,y_pred))
        print_confusion_matrix(confusion_matrix(y_test, y_pred), unique_labels(y_test, y_pred), model=clf)
        if inc_train == True:
            print(model_abrv[clf])
            print('\nTrain Stats\n', classification_report(y_train,clf_model.predict(X_train)))
            print_confusion_matrix(confusion_matrix(y_train, clf_model.predict(X_train)), unique_labels(y_test, y_pred), model=clf)
    if cv == True:
        print(model_abrv[clf] + ' CV Accuracy:',  
              np.mean(cross_val_score(clf_model, X_train, y_train, cv=5, scoring='accuracy')))
    if save == True:
        return clf_model

In [ ]:
%time
def model_train(model, train_data, train_labels):
  pipe = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,2), use_idf = True, tokenizer=lambda x: x.split(), max_features=20000)),  
        ('Transformer', TfidfTransformer()),  
        ('M', model)
    ])
  pipe.fit(train_data, train_labels)
  return pipe
trained_ml_models = []
for i in models.keys():
    tt = model_train(i,train_data, train_labels)
    trained_ml_models.append(tt)
    model(tt, cv= True, print_stat = True)  #set cv = True for cross-validation .Required much time
    

In [ ]:
tf = TfidfVectorizer(ngram_range=(1,2), use_idf = True, tokenizer=lambda x: x.split(),  max_features=20000)
tf.fit_transform(clean_text_data) ### Bit confusion arises here need to resolve
train_data = tf.transform(train_data)
test_data = tf.transform(test_data)

## Optimization with optuna

In [ ]:
!pip install optuna

     |████████████████████████████████| 308 kB 4.3 MB/s 
     |████████████████████████████████| 210 kB 40.9 MB/s 
     |████████████████████████████████| 81 kB 8.6 MB/s 
     |████████████████████████████████| 78 kB 6.4 MB/s 
     |████████████████████████████████| 150 kB 54.4 MB/s 
     |████████████████████████████████| 113 kB 53.8 MB/s 
     |████████████████████████████████| 49 kB 5.0 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=8aea918fe9d46f53f712409e62a98f585b36f1b8c00668bb241f49325c3ba050
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import optuna

### Logistic Regression

In [ ]:
def objective_lr(trial):

    params = { 
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'solver': trial.suggest_categorical('solver', ['newton-cg', 'sag', 'saga', 'lbfgs']),
        'multi_class':'multinomial',
        'max_iter': 5000
        }
    text_clf = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,1), use_idf = True, tokenizer=lambda x: x.split(), max_features=15000)),  
        ('Transformer', TfidfTransformer()),
       ( 'clf' ,LogisticRegression(**params, random_state = 22).set_params(**params))  
    
    
])

    text_clf.fit(train_data, train_labels)
    


    return np.mean(cross_val_score(text_clf,train_data, train_labels,cv =  scoring='accuracy'))

In [ ]:
study = optuna.create_study(direction='maximize') #opimize rapidly
study.optimize(objective_lr, n_trials=10)

[I 2022-03-24 04:29:56,896] A new study created in memory with name: no-name-cfd30abc-e6c4-4706-a543-0b788f7f3ed5
[I 2022-03-24 04:30:00,928] Trial 0 finished with value: 0.6986659924390185 and parameters: {'class_weight': None, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.6986659924390185.
[I 2022-03-24 04:30:03,712] Trial 1 finished with value: 0.6986659924390185 and parameters: {'class_weight': None, 'solver': 'newton-cg'}. Best is trial 0 with value: 0.6986659924390185.
[I 2022-03-24 04:30:08,243] Trial 2 finished with value: 0.6986659924390185 and parameters: {'class_weight': None, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.6986659924390185.
[I 2022-03-24 04:30:12,265] Trial 3 finished with value: 0.6986659924390185 and parameters: {'class_weight': None, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.6986659924390185.
[I 2022-03-24 04:30:14,535] Trial 4 finished with value: 0.6984028372600214 and parameters: {'class_weight': 'balanced', 'solver': 'sag'}. Best is tri

In [ ]:
text_clf = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,1), use_idf = True, tokenizer=lambda x: x.split(), max_features=20000)),  
        ('Transformer', TfidfTransformer()),
      
])
text_clf.fit(train_data, train_labels)




Pipeline(steps=[('FE',
                 TfidfVectorizer(max_features=20000,
                                 tokenizer=<function <lambda> at 0x7f68bb6fcf80>)),
                ('Transformer', TfidfTransformer())])

In [ ]:
text_clf = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,1), use_idf = True, tokenizer=lambda x: x.split(), max_features=20000)),  
        ('Transformer', TfidfTransformer()),
       ('clf', LogisticRegressionCV(class_weight='balanced', max_iter=400)),
])

text_clf.fit(train_data, train_labels)


In [ ]:
def objective_lr(trial):

    params = { 
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'solver': trial.suggest_categorical('solver', ['newton-cg', 'sag', 'saga', 'lbfgs']),
        'multi_class':'multinomial',
        'max_iter': 5000
        }
    text_clf = Pipeline([
        ('FE', TfidfVectorizer(ngram_range=(1,1), use_idf = True, tokenizer=lambda x: x.split(), max_features=20000)),  
        ('Transformer', TfidfTransformer()),
       ( 'clf' ,LogisticRegression(**params, random_state = 22).set_params(**params))  
    
    
])

    text_clf.fit(train_data, train_labels)
    


    return np.mean(cross_val_score(text_clf, cv=5, scoring='accuracy'))

In [ ]:
study = optuna.create_study(direction='maximize') #opimize rapidly
study.optimize(objective_lr, n_trials=10)

In [ ]:
def objective_mlp(trial):

    params = {
        'activation': trial.suggest_categorical('activation', ['logistic', 'tanh', 'relu']),
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'sgd', 'adam']),
        'hidden_layer_sizes':trial.suggest_int('hidden_layer_sizes', 100, 1500),
        'alpha': trial.suggest_uniform('alpha', 0.001, 0.99),
        'batch_size':trial.suggest_int('batch_size', 150, 300), 
        'learning_rate': trial.suggest_categorical('learning_rate', ['adaptive', 'constant', 'invscaling']),
        'max_iter': 1000
        }
  
    model = MLPClassifier(**params, random_state = 22) 
    
    model.set_params(**params)

    return np.mean(cross_val_score(model, train_data, train_labels, cv=5, scoring='accuracy'))

In [ ]:
study = optuna.create_study(direction='maximize') #optimizattion required huge time
study.optimize(objective_mlp, n_trials=15)

[I 2021-09-18 15:13:57,761] A new study created in memory with name: no-name-4dd592fc-e8e8-4012-bfb1-48fe10d1f351
